# The Project #
1. This is a project with minimal scaffolding. Expect to use the the discussion forums to gain insights! It’s not cheating to ask others for opinions or perspectives!
2. Be inquisitive, try out new things.
3. Use the previous modules for insights into how to complete the functions! You'll have to combine Pillow, OpenCV, and Pytesseract
4. There are hints provided in Coursera, feel free to explore the hints if needed. Each hint provide progressively more details on how to solve the issue. This project is intended to be comprehensive and difficult if you do it without the hints.

### The Assignment ###
Take a [ZIP file](https://en.wikipedia.org/wiki/Zip_(file_format)) of images and process them, using a [library built into python](https://docs.python.org/3/library/zipfile.html) that you need to learn how to use. A ZIP file takes several different files and compresses them, thus saving space, into one single file. The files in the ZIP file we provide are newspaper images (like you saw in week 3). Your task is to write python code which allows one to search through the images looking for the occurrences of keywords and faces. E.g. if you search for "pizza" it will return a contact sheet of all of the faces which were located on the newspaper page which mentions "pizza". This will test your ability to learn a new ([library](https://docs.python.org/3/library/zipfile.html)), your ability to use OpenCV to detect faces, your ability to use tesseract to do optical character recognition, and your ability to use PIL to composite images together into contact sheets.

Each page of the newspapers is saved as a single PNG image in a file called [images.zip](./readonly/images.zip). These newspapers are in english, and contain a variety of stories, advertisements and images. Note: This file is fairly large (~200 MB) and may take some time to work with, I would encourage you to use [small_img.zip](./readonly/small_img.zip) for testing.

Here's an example of the output expected. Using the [small_img.zip](./readonly/small_img.zip) file, if I search for the string "Christopher" I should see the following image:
![Christopher Search](images/small_project.png)
If I were to use the [images.zip](./readonly/images.zip) file and search for "Mark" I should see the following image (note that there are times when there are no faces on a page, but a word is found!):
![Mark Search](images/large_project.png)

Note: That big file can take some time to process - for me it took nearly ten minutes! Use the small one for testing.

In [7]:
from zipfile import ZipFile

from PIL import Image
from PIL import ImageDraw
import pytesseract
import cv2 as cv
import numpy as np
from IPython.display import display

# loading the face detection classifier
face_cascade = cv.CascadeClassifier('../images/haarcascade_frontalface_default.xml')

# Set the word_search and the file path and RUN!

f = "images/small_img.zip"

Word_search = 'Mark'


def Zip_list(fpath):
    """
    Returns a list of names of the contents. Just pass in the file path.
    """
    names = []
    with ZipFile(fpath) as myzip:
        for file in myzip.infolist():
            with myzip.open(file) as f:
                names.append(f.name)
                
    return names


def show_rects(faces, imgObj):
    """
    draws rectangles in the areas that models has reconized faces.
    Supply the image and the coordinates.
    """
    pil_img = imgObj.copy()

    drawing=ImageDraw.Draw(pil_img)
    
    for x,y,w,h in faces:
        drawing.rectangle((x,y,x+w,y+h), outline="black")
        
    display(pil_img)
    
    
def show_faces_CS(faces, imgObj):
    """
    It crops the area of the image that haarcascade_frontalface identified as faces and returns the 
    thumbline of the cropped images on a contact sheet.
    Supply the image and the coordinates.
    """   
    pil_img = imgObj.copy()
    
    crop_img=[]
    size = (90,90)
    
    for x,y,w,h in faces:
        crop_img.append((pil_img.crop((x,y,x+w,y+h))))
        
      
    contact_sheet = Image.new(crop_img[0].mode, (90 * 5, 90 * ((int(len(crop_img) / 6)) + 1 )))
    y = 0
    x = 0

    for img in crop_img:
        
        img.thumbnail(size)
        contact_sheet.paste(img, (x, y))
        
        if x + 90 == contact_sheet.width:
            y += 90
            x = 0
        else:
            x += 90
            
    display(contact_sheet)

In [ ]:
# Accumulate the images from the zip file
images_pil = []
for name in Zip_list(f):
     with ZipFile(f) as myzip:
        image = Image.open(myzip.open(name))
        images_pil.append({'image':image, 'name': name})

# Accumulate the images from the zip file
images_cv = []
for name in Zip_list(f):
     with ZipFile(f, 'r') as myzip:
        img_data = myzip.read(name)
        cv_img = cv.imdecode(np.frombuffer(img_data, np.uint8), 1)
        images_cv.append({'image':cv_img, 'name': name})

# Turn the image in grayscale/ extract the text from the image/ Search the word and if present run
# the face cascade and return the croped faces on a contact sheet.

for img in range(len(images_pil)):
    
    gray = cv.cvtColor(images_cv[img]['image'], cv.COLOR_BGR2GRAY)
    
    text = pytesseract.image_to_string(gray)
    
    if Word_search in text:
            
        faces = face_cascade.detectMultiScale(images_cv[img]['image'], 1.31,5)
            
        print("Results found in file {}".format(images_pil[img]['name']))
            
        if len(faces) > 0:
            
            show_faces_CS(faces, images_pil[img]['image'])
                
        else:
            print("No faces were found in this file.")
                
    else:
        print("The word was not mentioned in the file {}".format(img['name']))